## 針對 Single Error Correction(SEC)做 Synthesizer
### python(.py) to verilog(.v)
### 需要兩個LUT(l-LUT 和 r-LUT)

#### Define function: (Not including 2's complement range)

In [1]:
import math
import numpy as np


def decimal_to_binary(decimal_number):
    if decimal_number == 0:
        return "0"
    # 處理正數：直接轉換
    if decimal_number > 0:
        return bin(decimal_number)[2:]  # 移除 "0b" 前綴
       # 處理負數：自動計算所需位元數（2's complement）
    abs_value = abs(decimal_number)
    bits = abs_value.bit_length() + 1  # 負數多一個符號位元
    two_complement_value = (1 << bits) + decimal_number
    
    return bin(two_complement_value)[2:]  # 移除 "0b" 前綴

def bits_needed(number):
    return number.bit_length()


def bits_range(bits, signed=False):     ## 可選擇設定 有號數 或 無號數
    if bits <= 0:
        return "位元數必須為正整數！"
    
    if signed:
        # 二補數範圍: -2^(bits-1) 到 2^(bits-1)-1
        min_value = -(2**(bits - 1))
        max_value = 2**(bits - 1) - 1
    else:
        # 無符號範圍: 0 到 2^bits - 1
        min_value = 0
        max_value = 2**bits - 1

    return min_value, max_value


print("一個2補數所需要的位元數",bits_needed(120))
print("一bits數可表示的2補數的範圍",bits_range(bits_needed(120)))
print("一個2補數表達為二進制為:",decimal_to_binary(-8))

一個2補數所需要的位元數 7
一bits數可表示的2補數的範圍 (0, 127)
一個2補數表達為二進制為: 11000


### 選取特定的位元數:

In [4]:
### 輸入數值為data, ANcode的A為AN
### 乘完A後的數值為 input
data = (1 << 52)-1   ## 
A = 50861
input = data*A
bits = bits_needed(input)
print("AN資料的位元數:", bits)
print(bits_needed(A))
print(bits_needed(data))
print(bits_needed(bits))

AN資料的位元數: 68
16
52
7


### 產生1-bit AWE (Arithmetic-Weight Error)

In [5]:
AWE_1=[]
e1=0     ## 錯誤數量
e1_data=[]
h = [];   #位置
for i in range(0, bits):
    e1_data.insert(e1 , input + 2**i)
    AWE_1.insert(e1, 2**i)
    e1 = e1 + 1
    e1_data.insert(e1 , input - 2**i)
    AWE_1.insert(e1, -2**i)
    e1 = e1 + 1
    h.insert(i,i+1)

h_awe = [val for num in h for val in (num, -num)]
print(bits)
print(input)
print("產生1-bit AWE:",AWE_1)
print("算術錯誤的資料:", e1_data)
print(e1)
print(np.size(AWE_1))
print(h_awe)

68
229057580647690746195
產生1-bit AWE: [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 536870912, -536870912, 1073741824, -1073741824, 2147483648, -2147483648, 4294967296, -4294967296, 8589934592, -8589934592, 17179869184, -17179869184, 34359738368, -34359738368, 68719476736, -68719476736, 137438953472, -137438953472, 274877906944, -274877906944, 549755813888, -549755813888, 1099511627776, -1099511627776, 2199023255552, -2199023255552, 4398046511104, -4398046511104, 8796093022208, -8796093022208, 17592186044416, -17592186044416, 35184372088832, -35184372088832, 70368744177664, -70368744177664, 140737488355328, -14073748

In [6]:
# 計算對應元素相減
result = [a - b for a, b in zip(e1_data, AWE_1)]
# 顯示結果
print(result)
print(set(result))

[229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 229057580647690746195, 2290575806

### 產生SEC的模數:

In [7]:
def custom_mod(a, b):
    if b == 0:
        return "錯誤：除數不能為零！"
    # 計算商和餘數，修正餘數計算邏輯
    quotient = a // b
    remainder = a - b * quotient
    # 修正負餘數的情況
    if remainder > 0 and a < 0:
        remainder -= abs(b)
        quotient += 1
    return remainder

SEC_R = []
for m in range(0,e1):
    mod = custom_mod(e1_data[m], A)
    SEC_R.append(mod)
    
sec_r = []
for m in range(0,e1):
    mod = e1_data[m] % A
    sec_r.append(mod)

print("SEC的模數:\n",SEC_R)
print(h_awe)
print("兩種取餘數法是否相同:",sec_r == SEC_R)
print("產生1-bit AWE:\n",AWE_1)
print("SEC的大小:",np.size(SEC_R))
print("1-bit AWE 大小:",np.size(AWE_1))

SEC的模數:
 [1, 50860, 2, 50859, 4, 50857, 8, 50853, 16, 50845, 32, 50829, 64, 50797, 128, 50733, 256, 50605, 512, 50349, 1024, 49837, 2048, 48813, 4096, 46765, 8192, 42669, 16384, 34477, 32768, 18093, 14675, 36186, 29350, 21511, 7839, 43022, 15678, 35183, 31356, 19505, 11851, 39010, 23702, 27159, 47404, 3457, 43947, 6914, 37033, 13828, 23205, 27656, 46410, 4451, 41959, 8902, 33057, 17804, 15253, 35608, 30506, 20355, 10151, 40710, 20302, 30559, 40604, 10257, 30347, 20514, 9833, 41028, 19666, 31195, 39332, 11529, 27803, 23058, 4745, 46116, 9490, 41371, 18980, 31881, 37960, 12901, 25059, 25802, 50118, 743, 49375, 1486, 47889, 2972, 44917, 5944, 38973, 11888, 27085, 23776, 3309, 47552, 6618, 44243, 13236, 37625, 26472, 24389, 2083, 48778, 4166, 46695, 8332, 42529, 16664, 34197, 33328, 17533, 15795, 35066, 31590, 19271, 12319, 38542, 24638, 26223, 49276, 1585, 47691, 3170, 44521, 6340, 38181, 12680]
[1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12, 13,

### ${r}$-LUT:從餘數 ${r}$ 解回位置 ${l}$

In [8]:
#### SEC_AN Decoder
# 2025/4/7
#-------------------------------------
# Used to do Trade-off Algorithm
# input: remainder, output: location
Bw = bits
Bn = bits_needed(data)
BA = bits_needed(A)
B_location = bits_needed(bits)
#-------------------------------------
fn = 'SEC_rLUT' + str(bits_needed(data)) + "bits" +'.v'
f = open(fn, 'w')

f.write('// Product (AN) Code SEC r-LUT\n')
f.write('// ' + fn + '\n')
f.write('// Received remainder r, output single error location.\n')
f.write('module SEC_rLUT%dbits(r, l);\n'% (bits_needed(data)))
f.write('input \t[%d:0]\tr;\n' % (BA-1))
f.write('output\treg\tsigned\t[%d:0]\tl;\n' % (B_location))

f.write('always@(*) begin\n')
f.write('\tcase(r)\n')
for i in range(2*Bw):
    f.write("\t\t%d: l = %+d;\n" % (SEC_R[i] ,h_awe[i]))
f.write("\t\tdefault: l = 0;\n")
f.write('\tendcase\n')
f.write('end\n\n')

f.write('endmodule\n')
f.close();
print('Module file %s is generated.\n' % (fn))

Module file SEC_rLUT52bits.v is generated.



### ${l}$-LUT:從位置 ${l}$ 解回餘數 ${r}$

In [9]:
#### SEC_AN Decoder
# 2025/4/7
#-------------------------------------
# Used to do Trade-off Algorithm
# input: location, output: remainder
Bw = bits
Bn = bits_needed(data)
BA = bits_needed(A)
B_location = bits_needed(bits)
#-------------------------------------
fn = 'SEC_lLUT' + str(bits_needed(data)) + "bits" +'.v'
f = open(fn, 'w')

f.write('// Product (AN) Code SEC l-LUT\n')
f.write('// ' + fn + '\n')
f.write('// Received single error location l, output remainder r.\n')
f.write('module SEC_lLUT%dbits(l, r);\n'% (bits_needed(data)))
f.write('input\tsigned\t[%d:0]\tl;\n' % (B_location))
f.write('output\treg\t[%d:0]\tr;\n' % (BA-1))

f.write('always@(*) begin\n')
f.write('\tcase(l)\n')
for i in range(2*Bw):
    f.write("\t\t%d: r = %d;\n" % (h_awe[i] ,SEC_R[i]))
f.write("\t\tdefault: r = 0;\n")
f.write('\tendcase\n')
f.write('end\n\n')

f.write('endmodule\n')
f.close();
print('Module file %s is generated.\n' % (fn))

Module file SEC_lLUT52bits.v is generated.

